# Regional plots of XCO2 level 3 satellite data

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
# Single time to display
time = "2015-02"

# Range for annual mean
time_range = slice("2015", "2020")

# Variables to plot
variables = ["xco2", "xco2_stderr", "xco2_stddev", "xco2_nobs"]

# Mask
min_land_fraction = 0.5  # Use None to switch off

# Regions
regions = {
    "north_america": {"lon_slice": slice(-160, -60), "lat_slice": slice(10, 90)},
    "europe_africa": {"lon_slice": slice(-20, 80), "lat_slice": slice(-5, 60)},
    "asia": {"lon_slice": slice(70, 165), "lat_slice": slice(-15, 80)},
}

## Define request

In [ ]:
collection_id = "satellite-carbon-dioxide"
request = {
    "processing_level": ["level_3"],
    "variable": "xco2",
    "sensor_and_algorithm": "merged_obs4mips",
    "version": ["4_5"],
}

## Define functions to cache

In [ ]:
def convert_units(da):
    if da.name.endswith("_nobs"):
        return da

    with xr.set_options(keep_attrs=True):
        if da.name.startswith("xch4") and da.attrs["units"] != "ppb":
            da = da * 1.0e9
            da.attrs["units"] = "ppb"
        elif da.name.startswith("xco2") and da.attrs["units"] != "ppm":
            da = da * 1.0e6
            da.attrs["units"] = "ppm"
    return da


def mask_and_scale(ds, min_land_fraction):
    if min_land_fraction is not None:
        ds = ds.where(ds["land_fraction"] >= min_land_fraction)
    for var, da in ds.data_vars.items():
        if (fill_value := da.attrs.pop("fill_value", None)) is not None:
            da = da.where(da != fill_value.astype(da.dtype))
        ds[var] = convert_units(da)
    return ds

## Download and transform

In [ ]:
ds = download.download_and_transform(
    collection_id,
    request,
    transform_func=mask_and_scale,
    transform_func_kwargs={"min_land_fraction": min_land_fraction},
)
ds_time = ds.sel(time=time)
ds_annual = diagnostics.annual_weighted_mean(ds.sel(time=time_range))
with xr.set_options(keep_attrs=True):
    ds_annual_anomaly = ds_annual - diagnostics.time_weighted_mean(
        ds.sel(time=time_range)
    )

## Plot single-time maps

In [ ]:
for var in variables:
    da = ds_time[var]
    plot.projected_map(da, show_stats=False)
    plt.show()

    fig, axs = plt.subplots(
        1, len(regions), subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(20, 5)
    )
    for ax, (region, kwargs) in zip(axs.flatten(), regions.items()):
        da_region = utils.regionalise(da, **kwargs)
        plot.projected_map(da_region, ax=ax, show_stats=False)
        ax.set_title(region)
    plt.show()

## Plot annual mean maps

In [ ]:
plot_kwargs = {"col": "year", "col_wrap": 3}
for var in variables:
    da = ds_annual[var]
    plot.projected_map(da, **plot_kwargs)
    plt.show()

    for region, kwargs in regions.items():
        da_region = utils.regionalise(da, **kwargs)
        facet = plot.projected_map(da_region, **plot_kwargs)
        facet.fig.suptitle(region)
        plt.show()

## Plot annual anomalies

In [ ]:
plot_kwargs = {"col": "year", "col_wrap": 3}
for var in variables:
    da = ds_annual_anomaly[var]
    plot.projected_map(da, **plot_kwargs)
    plt.show()

    for region, kwargs in regions.items():
        da_region = utils.regionalise(da, **kwargs)
        facet = plot.projected_map(da_region, **plot_kwargs)
        facet.fig.suptitle(region)
        plt.show()